&nbsp;

# <center>Analysis Challenge Assignment 1</center>
### <center>Alex School Selction</center>
<center>Author: Simon Chen, Jianan Dingqian</center>
<center>Time: Sep 28, 2021</center>

#### **1. Criteria**


Alex wants to pursue a degree in Education. She wants to find the best school within U.S. with following requirements: <br>
1. Safety (low crime)
2. Urban (big city preferred)
3. Diversity (diverse culture)
4. Quality (decent educational program)

#### **2. Tasks**


1. Create a dataset with limited information(e.g., crime rate, admission rate) to meet criteria.
2. Keep record of code and descriptions to explain designed approach.
3. Create a rule to rank the schools in consideration of all of Alex criteria taken together. (Bonus)


#### **3. Import**


###### a. Crime data



In [3]:
# Import libraries.
import numpy as np
import pandas as pd
# Reading the crime data in 2015 from shareable link into Pandas.
crime_data = pd.read_csv('https://raw.githubusercontent.com/lmaosimon/ColumbiaCourseProjects/main/HUDK%204050/Analysis%20Challenge%20Assignment%201/Crime_2015.csv')
# Convert numbers read as object into float
crime_data["ViolentCrime"]= crime_data["ViolentCrime"].str.replace(',', '').astype(float)
crime_data["PropertyCrime"]= crime_data["PropertyCrime"].str.replace(',', '').astype(float)
crime_data["Burglary"]= crime_data["Burglary"].str.replace(',', '').astype(float)
crime_data["Theft"]= crime_data["Theft"].str.replace(',', '').astype(float)

###### b. College Scorecard data

In [5]:
# Reading College Scorecard data in pandas df.
college_data = pd.read_csv('https://raw.githubusercontent.com/lmaosimon/ColumbiaCourseProjects/main/HUDK%204050/Analysis%20Challenge%20Assignment%201/CollegeScorecard.csv')

#### **4. Cleaning and Processing data**



###### a. Crime data








In [6]:
# Then we rank the score to get a SafetyScore
# Since the reports number is the rates of reported offenses per 100,000 population.
# So we could just sum them up for each city to get the total rates by creating a new column rates.
crime_data['rates'] = crime_data.sum(axis=1)

# This table is now arranging cities from Hige crime rate to Low crime rate
crime_data = crime_data.sort_values(by='rates',ascending=False) 

# Then we rank the score to get a SafetyScore
safety_score = range(0,378)
crime_data['SafetyScore'] = safety_score

Extracting from the crime_data, we make a clean data. It's a new data frame called safety_score with MSA, state, city, and SafetyScore.

In [7]:
# Finally we make a new table called safety_score with MSA, state, city, and SafetyScore
Safety = crime_data[['MSA','State','City','SafetyScore']]
Safety

,MSA,State,City,SafetyScore
225,"Monroe, LA M.S.A.2",LA,Monroe,0
272,"Pueblo, CO M.S.A.",CO,Pueblo,1
140,"Hammond, LA M.S.A.",LA,Hammond,2
5,"Alexandria, LA M.S.A.",LA,Alexandria,3
4,"Albuquerque, NM M.S.A.",NM,Albuquerque,4
...,...,...,...,...
149,"Houston-The Woodlands-Sugar Land, TX M.S.A.3",TX,Houston,373
371,"Aguadilla-Isabela, Puerto Rico M.S.A.","Aguadilla-Isabela, Puerto Rico M.S.A.",Aguadilla,374
91,"Denver-Aurora-Lakewood, CO M.S.A.3, 4",CO,Denver,375
376,"San German, Puerto Rico M.S.A.","San German, Puerto Rico M.S.A.",San German,376


###### b. College Scorecard data

*Urban*

> *We chose schools that are located in cities.*



In [8]:
# Convert data we need into proper type
college_data["LOCALE"] = college_data["LOCALE"].astype("Int64")
# In CollegeScorecardDataDictionary, it explains meanings of number in LOCALE column
# 11 -- City: Large (population of 250,000 or more)
# 12 -- City: Midsize (population of at least 100,000 but less than 250,000)
# 12 -- City: Small (population less than 100,000)
# We first select schools in cities
collegedata1 = college_data[(college_data.LOCALE==11)|(college_data.LOCALE==12)|(college_data.LOCALE==13)]
# Demonstration
urban_college = collegedata1[['INSTNM','CITY','STABBR','ZIP']]
urban_college

,INSTNM,CITY,STABBR,ZIP
0,Alabama A & M University,Normal,AL,35762
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110
2,Amridge University,Montgomery,AL,36117-3553
3,University of Alabama in Huntsville,Huntsville,AL,35899
4,Alabama State University,Montgomery,AL,36104-0271
...,...,...,...,...
7372,Northeastern University Global Network,Boston,MA,02115
7376,Arizona College-Mesa,Mesa,AZ,85201
7378,Central Georgia Technical College,Warner Robins,GA,31088
7379,Barber School of Pittsburgh,Pittsburgh,PA,15220


*Diversity*

>*We chose different ethnicities' total share of enrollment of undergraduate degree-seeking students as a measurement.*


> *Culture varieties are brought from all ethnicities. It can be measured by calculating the standard deviation of all ethnicities's share of enrollment in each school. When the STD in certain school is high, it means the spread of each share is large, and with a total share of 1, it means there must exist some unbalance bewteen two or more groups. STD becomes lower when shares have closer amounts, brings more culture related events.*





In [9]:
# Based on urban schools, we select data of UGDS
ugds = collegedata1[['INSTNM','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI','UGDS_2MOR','UGDS_NRA','UGDS_UNKN']]

# To measure the standard deviation, we calculate the standard deviation of all enthic ugds
# With a smaller standard deviation, means the share of each ethnity is close, so more diverse, vise versa
collegedata2 = collegedata1.copy()
collegedata2['UGDS_STD']=ugds.std(axis=1)

# We select rows with Zero or NaN STDs as Invalid and give them a 0 as diversity score
nan_std = collegedata2[collegedata2['UGDS_STD'].isnull()]
zero_std = collegedata2[collegedata2['UGDS_STD']==0]
invalid_std = zero_std.append(nan_std)
invalid_std['DiversityScore'] = 0

# Now we select rows with valid STDs
valid_std = collegedata2[collegedata2['UGDS_STD']>0]
# Then we sort the value and give them a diversity score
# Since larger STD means less diversity, sorting in descending
valid_std = valid_std.sort_values(by='UGDS_STD',ascending=False)
diverse_score = range(1,3392)
valid_std['DiversityScore'] = diverse_score

# Add two data frames back together
college_div = valid_std.append(invalid_std)

# Demonstration of ranking from most diverse to least
diverse_rank = college_div.sort_values(by='DiversityScore', ascending=False)
diverse_rank = diverse_rank[['INSTNM','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI','UGDS_2MOR','UGDS_NRA','UGDS_UNKN','UGDS_STD','DiversityScore']]
diverse_rank

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,UGDS_STD,DiversityScore
970,Hawaii Pacific University,0.2824,0.0520,0.1326,0.1742,0.0043,0.0241,0.1769,0.1196,0.0340,0.091243,3391
533,Charles A Jones Career and Education Center,0.2413,0.2316,0.1434,0.1919,0.0446,0.0281,0.0775,0.0000,0.0417,0.092631,3390
966,Chaminade University of Honolulu,0.1754,0.0504,0.0628,0.3047,0.0075,0.1379,0.1363,0.0129,0.1121,0.093115,3389
245,California State University-East Bay,0.1843,0.1083,0.2546,0.2439,0.0016,0.0111,0.0572,0.0659,0.0732,0.094848,3388
5366,Los Angeles Film School,0.1655,0.2340,0.2349,0.0325,0.0036,0.0014,0.1465,0.0234,0.1583,0.096392,3387
...,...,...,...,...,...,...,...,...,...,...,...,...
1741,New England School of Law,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1795,Calvin Theological Seminary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1797,Michigan School of Professional Psychology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1809,Michigan State University-College of Law,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


*Quality*
> In order to define a decent educational program, we are taking the following types of data into consideration:
1. Percentage of degrees awarded in Education.
2. Certificate of less than one academic year in Education.
3. Certificate of at least one but less than two academic years in Education.
4. Associate degree in Education.
5. Award of more than two but less than four academic years in Education.

> We would like to introduce our criteria of quality score:
*   1.5* number of Certificate of less than one academic year in Education  
*   2.5* number of Certificate of at least one but less than two academic years in Education 
*   4* number of Award of more than two but less than four academic years in Education
*   number of Associate degree in Education
*   Sum the number above
*   Use the sum to time, 1 plus percentage of degrees awarded in Education

> The final score evaluates the quality of that school's educational program, higher points reflecting better quality.














In [10]:
# Based on college_div, we select columns with the desired information
edu_program = college_div[['INSTNM','PCIP13','CIP13CERT1','CIP13CERT2','CIP13CERT4','CIP13ASSOC']]
college_edu = college_div.copy()

# Calculate the total point of edu program
edu = edu_program.copy()
edu['rawPoint'] = edu['CIP13CERT1']*1.5 + edu['CIP13CERT2']*2.5 + edu['CIP13CERT4']*4 + edu['CIP13ASSOC']
awd_perc = edu['PCIP13']+1
edu['FinalPoint'] = edu['rawPoint']*awd_perc
college_edu['EduPoint'] = edu['FinalPoint']

# invalid edu
nan_edu = college_edu[college_edu['EduPoint'].isnull()]
zero_edu =  college_edu[college_edu['EduPoint']==0]
invalid_edu = zero_edu.append(nan_edu)
invalid_edu['QualityScore'] = 0

# valid edu
valid_edu = college_edu[college_edu['EduPoint']>0]
valid_edu = valid_edu.sort_values(by='EduPoint',ascending=True)
qual_score = range(1,710)
valid_edu['QualityScore'] = qual_score

# Add two data frames back together
edu_quality = valid_edu.append(invalid_edu)

# demonstration
edu_rank = edu_quality.sort_values(by='QualityScore', ascending=False)
edu_rank = edu_rank[['INSTNM','PCIP13','CIP13CERT1','CIP13CERT2','CIP13CERT4','CIP13ASSOC', 'EduPoint', 'QualityScore']]
edu_rank

,INSTNM,PCIP13,CIP13CERT1,CIP13CERT2,CIP13CERT4,CIP13ASSOC,EduPoint,QualityScore
908,Georgia Northwestern Technical College,0.0240,2.0,2.0,2.0,2.0,18.4320,709
525,Shasta Bible College and Graduate School,0.2222,0.0,2.0,2.0,2.0,18.3330,708
4184,Moraine Park Technical College,0.0182,2.0,2.0,2.0,2.0,18.3276,707
186,Pulaski Technical College,0.0101,2.0,2.0,2.0,2.0,18.1818,706
1936,University of Minnesota-Twin Cities,0.0481,2.0,2.0,2.0,0.0,16.7696,705
...,...,...,...,...,...,...,...,...
2862,Aultman College of Nursing and Health Sciences,0.0000,0.0,0.0,0.0,0.0,0.0000,0
4596,Fortis College-Mobile,0.0000,0.0,0.0,0.0,0.0,0.0000,0
6849,The Salon Professional Academy-Elgin,0.0000,0.0,0.0,0.0,0.0,0.0000,0
3800,Our Lady of the Lake University,0.1299,0.0,0.0,0.0,0.0,0.0000,0


#### **5. Results and Conclusion**

> In order to get the final list of schools meeting given requirements. We need to make a data frame that includes urban colleges, diversity score, quality score, and the location's safety score. These three scores need to make up and form a final list of 40 schools.

> As all scores now are listing in a ranking manner, we would like to transfer rank numbers in to a point out of 100, the distinction between two consecutive ranks is 100/(highest rank number). The participance of each type of score is weighting the same portion, which is 33%.





In [11]:
# Combine 3 requirements' ranking score on 1 dataframe
comb1 = edu_quality[['INSTNM','CITY','STABBR','ZIP','DiversityScore','QualityScore']]
comb2 = comb1.copy()
comb2['City&State'] = comb1['CITY']+comb1['STABBR']
comb3 = Safety.copy()
comb3['City&State'] = Safety['City']+Safety['State']
comb3 = comb3.drop_duplicates(subset='City&State')
comb_all = comb2.merge(comb3, on='City&State', copy=False)
comb_all_use = comb_all.copy()

# Calculate the Final Score
# Safety has total rank of 377
comb_all['Safety(100%)'] = 100-(377-comb_all_use['SafetyScore'])*(100/377)
# Diversity has total rank of 3391
comb_all['Diversity(100%)'] = 100-(3391-comb_all_use['DiversityScore'])*(100/3391)
# Quality has total rank of 709
comb_all['Quality(100%)'] = 100-(709-comb_all_use['QualityScore'])*(100/709)
# Total final each 33%
comb_all['FinalScore'] = comb_all['Safety(100%)']*0.33 + comb_all['Diversity(100%)']*0.33 + comb_all['Quality(100%)']*0.33

# Print the final list
all_final = comb_all [['INSTNM','City&State','Safety(100%)','Diversity(100%)','Quality(100%)','FinalScore']]
all_final = all_final.sort_values(by='FinalScore', ascending= False)
all_final_list = all_final.head(40)
all_final_list.reset_index(drop=True)

,INSTNM,City&State,Safety(100%),Diversity(100%),Quality(100%),FinalScore
0,Houston Community College,HoustonTX,98.938992,96.726629,86.036671,92.961757
1,Community College of Denver,DenverCO,99.469496,97.139487,67.136812,87.036112
2,Liberty University,LynchburgVA,92.572944,76.879976,93.370945,86.731876
3,Housatonic Community College,BridgeportCT,91.246684,85.815394,71.509168,82.028511
4,Carnegie Mellon University,PittsburghPA,83.023873,95.664996,67.277856,81.169019
5,Minneapolis Community and Technical College,MinneapolisMN,61.273210,87.850192,94.922426,80.535123
6,San Juan College,FarmingtonNM,60.477454,80.153347,98.589563,78.942720
7,Midwestern Career College,ChicagoIL,65.251989,96.107343,75.599436,78.196393
8,Capital Community College,HartfordCT,67.904509,92.096727,75.881523,77.841311
9,Strayer University-Global Region,WashingtonDC,70.291777,78.236508,85.331453,77.173714


Here above is the 40-school list in consideration of all Alex's criteria.